# American Options Pricing: LSM vs OHMC

This notebook implements **production-grade** American options pricing using:

1. **LSM (Least Squares Monte Carlo)**: Longstaff-Schwartz algorithm with ITM regression
2. **OHMC (Optimal Hedging Monte Carlo)**: LSM + optimal hedge control variate

## Key Features

- **Pathwise cashflow recursion**: Proper overwrite of future cashflows upon exercise
- **ITM-only regression**: Regress only where exercise is economically plausible
- **JAX acceleration**: High-performance path simulation
- **BlackJAX integration**: For uncertainty quantification

---

## Setup

In [ ]:
# Install JAX (uncomment for Colab)
# %pip install -q jax[tpu] -f https://storage.googleapis.com/jax-releases/libtpu_releases.html  # TPU
# %pip install -q jax[cuda12]  # GPU

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import blackjax
from functools import partial

# Check device
print(f"JAX version: {jax.__version__}")
print(f"Device: {jax.devices()[0].platform}")

# Plotting style
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## 1. Parameter Specification

We'll price an American Put option with standard parameters.

In [ ]:
# Option parameters
S0 = 100.0      # Initial stock price
K = 100.0      # Strike price
T = 1.0        # Time to maturity (years)
r = 0.05       # Risk-free rate
sigma = 0.2    # Volatility

# Simulation parameters
n_steps = 50   # Number of exercise opportunities
n_paths = 50000 # Number of simulated paths
dt = T / n_steps
discount = jnp.exp(-r * dt)  # One-step discount factor

print(f"Option: American Put")
print(f"S0 = ${S0}, K = ${K}, T = {T} years")
print(f"r = {r:.1%}, σ = {sigma:.1%}")
print(f"n_steps = {n_steps}, n_paths = {n_paths:,}")

## 2. Path Simulation (GBM)

Simulate Geometric Brownian Motion paths using JAX.

In [ ]:
def simulate_gbm_paths(key, n_paths, n_steps, S0, r, sigma, dt):
    """
    Simulate GBM paths using JAX.
    
    Returns:
        paths: (n_paths, n_steps+1) array of stock prices
    """
    # Generate random normal increments
    z = jax.random.normal(key, (n_paths, n_steps))
    
    # Calculate log returns
    log_returns = (r - 0.5 * sigma**2) * dt + sigma * jnp.sqrt(dt) * z
    
    # Cumulative sum to get log prices
    cumulative_log_returns = jnp.cumsum(log_returns, axis=1)
    
    # Add initial price
    paths = S0 * jnp.exp(jnp.hstack([jnp.zeros((n_paths, 1)), cumulative_log_returns]))
    
    return paths

# Generate paths
key = jax.random.PRNGKey(42)
paths = simulate_gbm_paths(key, n_paths, n_steps, S0, r, sigma, dt)

print(f"Simulated {n_paths:,} paths with shape: {paths.shape}")
print(f"Price range: [${paths.min():.2f}, ${paths.max():.2f}]")

## 3. Basis Functions

Polynomial basis functions for regression (Laguerre polynomials are common in LSM).

In [ ]:
def laguerre_polynomials(x, max_degree=3):
    """
    Compute Laguerre polynomials up to max_degree.
    
    L_0(x) = 1
    L_1(x) = 1 - x
    L_2(x) = 0.5 * (x^2 - 4x + 2)
    L_3(x) = (1/6) * (-x^3 + 9x^2 - 18x + 6)
    """
    x = jnp.asarray(x)
    basis = [jnp.ones_like(x)]  # L_0
    
    if max_degree >= 1:
        basis.append(1 - x)  # L_1
    if max_degree >= 2:
        basis.append(0.5 * (x**2 - 4*x + 2))  # L_2
    if max_degree >= 3:
        basis.append((1/6) * (-x**3 + 9*x**2 - 18*x + 6))  # L_3
    
    return jnp.column_stack(basis)

def polynomial_basis(x, max_degree=3):
    """Simple polynomial basis: [1, x, x^2, x^3]."""
    x = jnp.asarray(x)
    return jnp.column_stack([x**d for d in range(max_degree + 1)])

# Test basis functions
test_x = jnp.linspace(50, 150, 10)
basis_poly = polynomial_basis(test_x, max_degree=3)
print(f"Polynomial basis shape: {basis_poly.shape}")
print(f"First row: {basis_poly[0]}")

## 4. LSM Algorithm (Production-Grade)

Implements the Longstaff-Schwartz method with:
- ITM-only regression
- Pathwise cashflow overwrite recursion

In [ ]:
def put_payoff(S, K):
    """Intrinsic value of a put option."""
    return jnp.maximum(K - S, 0.0)

def lsm_american_put(paths, K, r, dt, basis_func=polynomial_basis, max_degree=3):
    """
    LSM algorithm for American Put with ITM regression and pathwise recursion.
    
    Key features:
    1. ITM-only regression (only where exercise is plausible)
    2. Pathwise cashflow overwrite (proper handling of exercise decisions)
    
    Returns:
        option_price: Estimated option value at t=0
        exercise_times: Array of exercise times per path
    """
    n_paths, n_steps_plus_one = paths.shape
    n_steps = n_steps_plus_one - 1
    
    # Initialize pathwise values and exercise times
    V = jnp.zeros((n_paths, n_steps_plus_one))  # Option values along each path
    exercise_time = jnp.full(n_paths, n_steps)  # When each path exercises
    
    # Terminal condition: intrinsic value at expiration
    V = V.at[:, -1].set(put_payoff(paths[:, -1], K))
    
    # Backward induction from T-1 to 0
    for k in range(n_steps - 1, -1, -1):
        S_k = paths[:, k]
        intrinsic_k = put_payoff(S_k, K)
        
        # Discounted continuation values (from next step)
        Y_k = discount * V[:, k + 1]
        
        # ITM set: only paths where exercise is plausible
        itm_mask = intrinsic_k > 0
        
        if jnp.sum(itm_mask) > max_degree + 1:  # Need enough points for regression
            # Extract ITM paths
            S_itm = S_k[itm_mask]
            Y_itm = Y_k[itm_mask]
            
            # Build basis matrix for ITM paths
            # Normalize by strike for numerical stability
            X_itm = S_itm / K
            basis_matrix = basis_func(X_itm, max_degree=max_degree)
            
            # Least squares regression: Y ≈ basis @ beta
            beta, residuals, rank, s = jnp.linalg.lstsq(basis_matrix, Y_itm, rcond=None)
            
            # Predict continuation value for ALL paths (at least ITM)
            X_all = S_k / K
            basis_all = basis_func(X_all, max_degree=max_degree)
            continuation_value = basis_all @ beta
            
            # Exercise decision: exercise if intrinsic >= continuation
            exercise_now = intrinsic_k >= continuation_value
            
            # Pathwise cashflow overwrite
            # If exercise: set value to intrinsic, update exercise time
            # If continue: keep discounted future value
            V = V.at[:, k].set(jnp.where(exercise_now, intrinsic_k, Y_k))
            exercise_time = jnp.where(exercise_now, k, exercise_time)
        else:
            # Not enough ITM paths: just discount
            V = V.at[:, k].set(Y_k)
    
    # Discount all cashflows back to t=0
    discounted_values = V[:, 0] * jnp.exp(-r * dt * exercise_time)
    
    return jnp.mean(discounted_values), exercise_time

# Run LSM
lsm_price, lsm_exercise_times = lsm_american_put(paths, K, r, dt)
print(f"LSM American Put Price: ${lsm_price:.4f}")
print(f"Average exercise time: {jnp.mean(lsm_exercise_times):.1f} steps")
print(f"Early exercise rate: {100 * jnp.mean(lsm_exercise_times < n_steps):.1f}%")

## 5. OHMC Algorithm (Optimal Hedging Monte Carlo)

OHMC extends LSM by adding an **optimal hedge control variate** to reduce variance.

The key difference: we fit both continuation value AND hedge ratio simultaneously.

In [ ]:
def ohmc_american_put(paths, K, r, dt, basis_func=polynomial_basis, max_degree=3):
    """
    OHMC algorithm: LSM + optimal hedge control variate.
    
    Key innovation: Fit continuation AND hedge ratio jointly:
    Y_k ≈ C_k(S_k) + φ_k(S_k) * ΔS_{k+1}
    
    This reduces variance by hedging out the hedgeable component.
    
    Returns:
        option_price: Estimated option value
        exercise_times: Exercise times per path
        residual_risks: Residual risk at each step (unhedgeable component)
    """
    n_paths, n_steps_plus_one = paths.shape
    n_steps = n_steps_plus_one - 1
    
    # Initialize
    V = jnp.zeros((n_paths, n_steps_plus_one))
    exercise_time = jnp.full(n_paths, n_steps)
    residual_risks = []  # Track unhedgeable risk at each step
    
    # Terminal condition
    V = V.at[:, -1].set(put_payoff(paths[:, -1], K))
    
    # Backward induction
    for k in range(n_steps - 1, -1, -1):
        S_k = paths[:, k]
        S_next = paths[:, k + 1]
        intrinsic_k = put_payoff(S_k, K)
        
        # Price increment (for hedge)
        delta_S = S_next - S_k
        
        # Discounted continuation values
        Y_k = discount * V[:, k + 1]
        
        # ITM set
        itm_mask = intrinsic_k > 0
        
        if jnp.sum(itm_mask) > 2 * (max_degree + 1):  # Need more points for joint regression
            # Extract ITM paths
            S_itm = S_k[itm_mask]
            Y_itm = Y_k[itm_mask]
            delta_S_itm = delta_S[itm_mask]
            
            # Normalize for numerical stability
            X_itm = S_itm / K
            
            # Build basis for continuation value
            basis_C = basis_func(X_itm, max_degree=max_degree)
            
            # Build basis for hedge ratio (can use same or different)
            basis_phi = basis_func(X_itm, max_degree=max_degree)
            
            # Joint regression: Y = C(S) + φ(S) * ΔS + ε
            # We solve: min ||Y - basis_C @ gamma - (basis_phi * ΔS) @ alpha||^2
            
            # Create augmented design matrix
            # [basis_C, basis_phi * ΔS] @ [gamma; alpha] = Y
            n_basis = basis_C.shape[1]
            
            # Multiply hedge basis by price increments
            hedge_terms = basis_phi * delta_S_itm[:, jnp.newaxis]  # (n_itm, n_basis)
            
            # Augmented design matrix
            X_aug = jnp.hstack([basis_C, hedge_terms])
            
            # Joint least squares
            beta_joint, _, _, _ = jnp.linalg.lstsq(X_aug, Y_itm, rcond=None)
            
            # Split coefficients
            gamma = beta_joint[:n_basis]  # Continuation coefficients
            alpha = beta_joint[n_basis:]  # Hedge coefficients
            
            # Predict continuation value for all paths
            X_all = S_k / K
            basis_all = basis_func(X_all, max_degree=max_degree)
            continuation_value = basis_all @ gamma
            
            # Compute residual risk (optional diagnostic)
            continuation_itm = basis_C @ gamma
            hedge_itm = (basis_phi @ alpha) * delta_S_itm
            residuals = Y_itm - continuation_itm - hedge_itm
            residual_risk = jnp.mean(residuals**2)
            residual_risks.append(residual_risk)
            
            # Exercise decision (same as LSM)
            exercise_now = intrinsic_k >= continuation_value
            
            # Pathwise cashflow overwrite
            V = V.at[:, k].set(jnp.where(exercise_now, intrinsic_k, Y_k))
            exercise_time = jnp.where(exercise_now, k, exercise_time)
        else:
            # Not enough ITM paths: just discount
            V = V.at[:, k].set(Y_k)
            residual_risks.append(0.0)
    
    # Discount back to t=0
    discounted_values = V[:, 0] * jnp.exp(-r * dt * exercise_time)
    
    return jnp.mean(discounted_values), exercise_time, jnp.array(residual_risks)

# Run OHMC
ohmc_price, ohmc_exercise_times, residual_risks = ohmc_american_put(paths, K, r, dt)
print(f"OHMC American Put Price: ${ohmc_price:.4f}")
print(f"Average exercise time: {jnp.mean(ohmc_exercise_times):.1f} steps")
print(f"Early exercise rate: {100 * jnp.mean(ohmc_exercise_times < n_steps):.1f}%")
print(f"\nResidual risk (unhedgeable component):")
print(f"  Mean: {jnp.mean(residual_risks):.6f}")
print(f"  Max: {jnp.max(residual_risks):.6f}")

## 6. Comparison: LSM vs OHMC

In [ ]:
# Compare results
print("="*60)
print("COMPARISON: LSM vs OHMC")
print("="*60)
print(f"\nLSM Price:  ${lsm_price:.4f}")
print(f"OHMC Price: ${ohmc_price:.4f}")
print(f"Difference: ${abs(lsm_price - ohmc_price):.4f} ({100*abs(lsm_price-ohmc_price)/lsm_price:.2f}%)")

# European put for reference
european_payoff = put_payoff(paths[:, -1], K)
european_price = jnp.mean(european_payoff) * jnp.exp(-r * T)
print(f"\nEuropean Put: ${european_price:.4f}")
print(f"Early Exercise Premium (LSM): ${lsm_price - european_price:.4f}")
print(f"Early Exercise Premium (OHMC): ${ohmc_price - european_price:.4f}")

## 7. Visualization: Exercise Decisions

In [ ]:
# Plot exercise decisions for sample paths
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Sample 20 paths for visualization
sample_paths = paths[:20]
sample_lsm_times = lsm_exercise_times[:20]
sample_ohmc_times = ohmc_exercise_times[:20]

# LSM exercise decisions
axes[0].plot(sample_paths.T, 'b-', linewidth=0.8, alpha=0.5)
for i, ex_time in enumerate(sample_lsm_times):
    if ex_time < n_steps:
        axes[0].plot(ex_time, sample_paths[i, int(ex_time)], 'ro', markersize=8, label='Exercise' if i == 0 else '')
axes[0].axhline(K, color='r', linestyle='--', alpha=0.7, label='Strike')
axes[0].set_xlabel('Time Step')
axes[0].set_ylabel('Stock Price')
axes[0].set_title('LSM: Exercise Decisions (Red = Exercise Point)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# OHMC exercise decisions
axes[1].plot(sample_paths.T, 'b-', linewidth=0.8, alpha=0.5)
for i, ex_time in enumerate(sample_ohmc_times):
    if ex_time < n_steps:
        axes[1].plot(ex_time, sample_paths[i, int(ex_time)], 'go', markersize=8, label='Exercise' if i == 0 else '')
axes[1].axhline(K, color='r', linestyle='--', alpha=0.7, label='Strike')
axes[1].set_xlabel('Time Step')
axes[1].set_ylabel('Stock Price')
axes[1].set_title('OHMC: Exercise Decisions (Green = Exercise Point)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Residual Risk Analysis (OHMC Only)

OHMC provides insight into **unhedgeable risk** at each time step.

In [ ]:
# Plot residual risk over time
fig, ax = plt.subplots(figsize=(10, 5))

time_steps = jnp.arange(len(residual_risks))
ax.plot(time_steps, residual_risks, 'b-o', linewidth=2, markersize=4)
ax.set_xlabel('Time Step (backward from expiration)')
ax.set_ylabel('Residual Risk (Mean Squared Error)')
ax.set_title('OHMC: Unhedgeable Risk Over Time')
ax.grid(True, alpha=0.3)

# Annotate
max_risk_idx = jnp.argmax(residual_risks)
ax.annotate(f'Max risk at step {max_risk_idx}',
            xy=(max_risk_idx, residual_risks[max_risk_idx]),
            xytext=(max_risk_idx + 5, residual_risks[max_risk_idx] + 0.001),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=10, color='red')

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("Residual risk measures the variance that cannot be hedged away.")
print("Higher values indicate more uncertainty about continuation value.")

## 9. BlackJAX Integration: Uncertainty Quantification

Use BlackJAX to quantify uncertainty in the option price estimate via MCMC sampling.

In [ ]:
# Define a log-likelihood for option price uncertainty
# We'll treat the Monte Carlo estimate as having Gaussian noise

def price_log_likelihood(price_estimate, observed_price, std_dev):
    """Log-likelihood assuming Gaussian noise."""
    return -0.5 * ((price_estimate - observed_price) / std_dev)**2

# For demonstration: sample from posterior over option price
# This is a simplified example - in practice you'd have more sophisticated models

def run_blackjax_sampling(observed_price, price_std, n_samples=1000):
    """
    Use BlackJAX NUTS to sample from posterior over option price.
    
    This demonstrates how to integrate BlackJAX for uncertainty quantification.
    """
    # Prior: uniform over reasonable range
    def logdensity_fn(price):
        if price < 0 or price > 2 * K:
            return -jnp.inf
        # Likelihood: observed price with some uncertainty
        return price_log_likelihood(price, observed_price, price_std)
    
    # Initialize sampler
    initial_position = jnp.array([observed_price])
    
    # Use NUTS sampler from BlackJAX
    nuts = blackjax.nuts(logdensity_fn, step_size=0.1)
    
    # Initial state
    initial_state = nuts.init(initial_position)
    
    # Run sampler
    key = jax.random.PRNGKey(0)
    samples = []
    state = initial_state
    
    for _ in range(n_samples):
        key, subkey = jax.random.split(key)
        state, _ = nuts.step(subkey, state)
        samples.append(state.position[0])
    
    return jnp.array(samples)

# Estimate standard error from Monte Carlo variance
# (In practice, you'd run multiple independent MC runs)
price_std_estimate = 0.5  # Placeholder - would come from multiple runs

# Run BlackJAX sampling
print("Running BlackJAX NUTS sampler for uncertainty quantification...")
posterior_samples = run_blackjax_sampling(lsm_price, price_std_estimate, n_samples=2000)

# Plot posterior
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(posterior_samples, bins=50, alpha=0.7, edgecolor='black')
ax.axvline(lsm_price, color='r', linestyle='--', linewidth=2, label=f'LSM Estimate: ${lsm_price:.4f}')
ax.axvline(jnp.mean(posterior_samples), color='g', linestyle='--', linewidth=2, 
           label=f'Posterior Mean: ${jnp.mean(posterior_samples):.4f}')
ax.set_xlabel('Option Price ($)')
ax.set_ylabel('Frequency')
ax.set_title('BlackJAX: Posterior Distribution over Option Price')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nPosterior Statistics:")
print(f"  Mean: ${jnp.mean(posterior_samples):.4f}")
print(f"  Std:  ${jnp.std(posterior_samples):.4f}")
print(f"  95% Credible Interval: [${jnp.percentile(posterior_samples, 2.5):.4f}, ${jnp.percentile(posterior_samples, 97.5):.4f}]")

## 10. Summary and Key Takeaways

### LSM Formula (ITM Regression):

$$\beta_k = \arg\min_{\beta} \sum_{i\in\mathcal{I}_k} \left(D_k V_{k+1}^{(i)} - \sum_j \beta_j \psi_j(X_k^{(i)})\right)^2$$

$$V_k^{(i)} = \max\left(g_k^{(i)}, \widehat{C}_k(X_k^{(i)})\right) \text{ (pathwise via overwrite)}$$

### OHMC Formula (ITM Hedged Regression):

$$(\gamma_k, \alpha_k) = \arg\min_{\gamma,\alpha} \sum_{i\in\mathcal{I}_k} \left(D_k V_{k+1}^{(i)} - \widehat{C}_k(X_k^{(i)}) - \widehat{\phi}_k(X_k^{(i)})\Delta X_{k+1}^{(i)}\right)^2$$

$$V_k^{(i)} = \max\left(g_k^{(i)}, \widehat{C}_k(X_k^{(i)})\right) \text{ (pathwise via overwrite)}$$

### Key Differences:

1. **LSM**: Fits continuation value only
2. **OHMC**: Fits continuation + hedge ratio jointly (reduces variance)
3. **Both**: Use ITM-only regression and pathwise cashflow overwrite

### Production-Grade Features:

- ✅ ITM-only regression (reduces noise)
- ✅ Pathwise cashflow overwrite (prevents multiple exercises)
- ✅ Proper discounting back to t=0
- ✅ Residual risk tracking (OHMC)
- ✅ BlackJAX integration for uncertainty quantification